What is EXIF Data? 

When viewing images on the Internet, you might have noticed that some websites provide valuable information related to those images, such as their exposure settings, camera brand, etc. This information known as “EXIF Data” can be an important source of knowledge in figuring out how photographers capture images and what tools they use in the process. In Python, we can view exif data with the Python module called exifread. The word “EXIF” is based on the Exchangeable Image File Format standard and data that photographers care about such as shutter speed, aperture, date, etc are considered EXIF data. Nowadays, every modern digital camera has the capability to record this information, along with many other camera settings, right into photographs. These settings can then be later used to organize photographs, perform searches and provide vital information to photographers about the way a particular photograph was captured. What we can do with EXIF data is exploit images to get GPS locations.

In [ ]:
#First, let's install exifread

!pip install exifread

Import exifread along with a few other modules to handle inspecting this data.

To read more about these modules see below:
os: https://docs.python.org/3.7/library/os.html#module-os
os.path(): https://docs.python.org/3.7/library/os.path.html
glob: https://docs.python.org/3.7/library/glob.html#module-glob
pprint: https://docs.python.org/3.7/library/pprint.html#module-pprint

In [ ]:
import os
import glob
import pprint
import exifread

We'll need some sample data to process EXIF tags on. You may have a batch of photos but for consistency let's use the material from here so we can expect the same results.

https://github.com/ianare/exif-samples

https://github.com/ianare/exif-samples/archive/master.zip

The below assumes you've extracted this zip file under the current working directory.  Let's see what directory it is so we can make sure we dropped the zip file into the write directory/folder.

In [ ]:
#Let's see what the path of our current working directory is

os.getcwd()

In [ ]:
cwd = os.path.abspath('.')

if not os.path.exists(os.path.join(cwd, 'exif-samples-master')):
    print("exif-samples-master not found")

With the files in place, we can inspect the dictionary dump of the results. This produces a lot of results so pprint will come in handed here.

In [ ]:
for filename in glob.glob('exif-samples-master/petra/*.jpg', recursive=True):
    
    # Open image file for reading (binary mode)
    fh = open(os.path.join(cwd, filename), 'rb')

    # Return Exif tags
    tags = exifread.process_file(fh)
    
    pprint.pprint(tags)

You can see that there's some very cool data that we can pull out from this. However, because we're good with opsec, we all had location data turned off on our phones.  Let's try to pull some GPS coordinates that are embedded in the EXIF data of the other pictures in the master folder.  We don't have to reinvent the wheel here and can use material that is out there publicly. This is from https://gist.github.com/snakeye/fdc372dbf11370fe29eb.

In [ ]:
# https://gist.github.com/snakeye/fdc372dbf11370fe29eb

def _get_if_exist(data, key):
    if key in data:
        return data[key]

    return None


def _convert_to_degress(value):
    """
    Helper function to convert the GPS coordinates stored in the EXIF to degress in float format
    :param value:
    :type value: exifread.utils.Ratio
    :rtype: float
    """
    d = float(value.values[0].num) / float(value.values[0].den)
    m = float(value.values[1].num) / float(value.values[1].den)
    s = float(value.values[2].num) / float(value.values[2].den)

    return d + (m / 60.0) + (s / 3600.0)
    
def get_exif_location(exif_data):
    """
    Returns the latitude and longitude, if available, from the provided exif_data (obtained through get_exif_data above)
    """
    lat = None
    lon = None

    gps_latitude = _get_if_exist(exif_data, 'GPS GPSLatitude')
    gps_latitude_ref = _get_if_exist(exif_data, 'GPS GPSLatitudeRef')
    gps_longitude = _get_if_exist(exif_data, 'GPS GPSLongitude')
    gps_longitude_ref = _get_if_exist(exif_data, 'GPS GPSLongitudeRef')

    if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
        lat = _convert_to_degress(gps_latitude)
        if gps_latitude_ref.values[0] != 'N':
            lat = 0 - lat

        lon = _convert_to_degress(gps_longitude)
        if gps_longitude_ref.values[0] != 'E':
            lon = 0 - lon

    return lat, lon

With the above defined, let's try some functionality to inspect our results.

In [ ]:
for filename in glob.glob('exif-samples-master/**/*.jpg', recursive=True):
    # Open image file for reading (binary mode)
    fh = open(filename, 'rb')

    try:
        # Return Exif tags
        tags = exifread.process_file(fh)
    
        gps_tuple = get_exif_location(tags)
    
        if gps_tuple != (None, None):
            print(filename, gps_tuple)

    # there are known corrupt files, we must handle this
    except OSError:
        pass
    except ZeroDivisionError:
        pass

With the above worked out, let's try to visualize these results more effectively.

In [ ]:
!pip install folium

In [ ]:
import folium

Let's center on the Eiffel Tower in Paris since we have multiple images with Europe EXIF data

In [ ]:
map_ = folium.Map(location=[48.8584, 2.2945], zoom_start=3)

Try it out! Swap the print of lat/lon for marking up a map with.
Essentially, we are inserting the gps_tuple coordinates into the folium.Marker expression

In [ ]:
for filename in glob.glob('exif-samples-master/**/*.jpg', recursive=True):
    # Open image file for reading (binary mode)
    fh = open(filename, 'rb')

    try:
        # Return Exif tags
        tags = exifread.process_file(fh)
    
        gps_tuple = get_exif_location(tags)
    
        if gps_tuple != (None, None):
            # print(filename, gps_tuple)
            folium.Marker(gps_tuple).add_to(map_)

    # there are known corrupt files, we must handle this
    except OSError:
        pass
    except ZeroDivisionError:
        pass

Display it!

In [ ]:
map_

In [ ]:
#let's try to use simplekml
!pip install simplekml

In [ ]:
import simplekml

In [ ]:
# kml = simplekml.Kml()
# df.apply(lambda X: kml.newpoint(name=X["name"], coords=[(X["longitude"], X["latitude"])]), axis=1)
# kml.save(path="data.kml")

In [ ]:
kml = simplekml.Kml()
gpslist = []
for filename in glob.glob('exif-samples-master/**/*.jpg', recursive=True):
    # Open image file for reading (binary mode)
    fh = open(filename, 'rb')

    try:
        # Return Exif tags
        tags = exifread.process_file(fh)
    
        gps_tuple = get_exif_location(tags)
    
        if gps_tuple != (None, None):
            gpslist.append(gps_tuple)

    # there are known corrupt files, we must handle this
    except OSError:
        pass
    except ZeroDivisionError:
        pass

In [ ]:
gpslist

Again, we don't need to reinvent the wheel.  I went to the simplekml website and took the generic code for how to make a kml and turn our gps points into coordinates.  Note in the code how Google Earth takes the inputs of the coordinates as Longitude and then Latitude.  

https://simplekml.readthedocs.io/en/latest/tut_point.html

In [ ]:
# Create an instance of Kml
kml = simplekml.Kml(open=1)

# Create a point named "The World" attached to the KML document with its coordinate at 0 degrees latitude and longitude.
# All the point's properties are given when it is constructed.
#single_point = kml.newpoint(name="The World", coords=[(0.0,0.0)])

# Create a point for each city. The points' properties are assigned after the point is created
for lat, lon in gpslist:
    pnt = kml.newpoint()
    pnt.coords = [(lon, lat)]
    

# Save the KML
kml.save("exifdata.kml")
print('We made the kml for you!')